# Created a Merged Station Inventory for GAIA / CRESST

Precip stations https://docs.synopticdata.com/services/metadata

Requires an academic account + token for interfacing with API 

In [ ]:
import requests
import os
import pandas as pd
import geopandas as gpd

In [ ]:
TOKEN = os.environ.get('SYNOPTIC_TOKEN')

In [ ]:
# List of stations
baseAPI = "https://api.synopticdata.com/v2"
endpoint = "/stations/metadata"
url = baseAPI + endpoint

params = dict(state="wa",
              token=TOKEN,
)

response = requests.get(url, params=params)

data = response.json()

In [ ]:
data['SUMMARY']

In [ ]:
df = pd.json_normalize(data['STATION'])

In [ ]:
df.head()

In [ ]:
df.iloc[0]

## Precipitation

In [ ]:
# filter search to stations with precip data
# https://demos.synopticdata.com/variables/index.html
precip_vars = ["precip_accum"] # NOTE: there are a lot! e.g. hourly etc
sensorvars=True

params = dict(state="wa",
              token=TOKEN,
              sensorvars=True, # return sensor variable info
              vars=','.join(precip_vars), # restrict to precip only
              output='geojson',
)
response = requests.get(url, params=params)
data = response.json()


In [ ]:
gfp = gpd.GeoDataFrame.from_features(data['features'], crs='EPSG:4326')
gfp.head()

In [ ]:
# A rogue statio in Mexico!
gfp = gfp[gfp.stid != 'E0744']

#gfp.explore(column='status',popup=True, cmap=['green','red'])

In [ ]:
#gfp.to_file('precip-stations-wa.geojson', driver='GeoJSON')
# For starters drop Inactive and restricted sensors
keep_cols = ['geometry','id','stid','name','elevation','status','period_of_record','station_info','restricted_data']
gfp = gfp[(gfp.status == 'ACTIVE') & (gfp.restricted_data == False)][keep_cols]
print(len(gfp), "active, unrestricted precip stations in WA")
gfp.head()

In [ ]:
# Add separate columns for period_of_record start and end
gfp['start_datetime'] = pd.to_datetime(gfp['period_of_record'].apply(lambda x: x['start']))
gfp['end_datetime'] = pd.to_datetime(gfp['period_of_record'].apply(lambda x: x['end']))
new_order = ["id", "stid", "name", "start_datetime", "end_datetime", "elevation", "station_info", "geometry"]
gfp = gfp[new_order]
gfp.head()

In [ ]:
# Add simple-styling for geojson.io
# https://github.com/mapbox/simplestyle-spec/blob/master/1.1.0/README.md

gfp['marker-color'] = '#0000FF' # blue
gfp['marker-size'] = 'small'
gfp['marker-symbol'] = 'water'

# Apparently these symbols are supported:
# https://labs.mapbox.com/maki-icons/

# # Just for illustration purposes!
# def style_marker(row):
#     if row['status'] == 'ACTIVE':
#         #color = '#00FF00' # light green
#         size = 'medium'
#         symbol = 'water'
#     else:
#         #color = '#006400' # dark green
#         size = 'small'
#         symbol = 'water'

#     # Or single color and change stroke / for active or not:
#     color = '#0000FF' # blue

#     return pd.Series({'marker-color': color, 'marker-size': size, 'marker-symbol': symbol})

# gfp[['marker-color','marker-size','marker-symbol']] = gfp.apply(style_marker, axis=1)
# gfp.head()

In [ ]:
# Save simplified, styled geojson
gfp.to_file("precip-stations-wa-styled.geojson", driver='GeoJSON')
!ls -tlrh *geojson

## Streamflow

In [ ]:
# Search USGS River Gauges:
# filter search to stations with precip data
# https://demos.synopticdata.com/variables/index.html
target_vars = ["stream_flow"]
sensorvars=True

params = dict(state="wa",
              token=TOKEN,
              #sensorvars=True, # Doesn't seem to work for geojson output...
              vars=','.join(target_vars),
              output='geojson',
)
response = requests.get(url, params=params)
data = response.json()


In [ ]:
gfs = gpd.GeoDataFrame.from_features(data['features'], crs='EPSG:4326')
print('stations=', len(gfs))
gfs.head()

In [ ]:
# Only ACTIVE sensores Add separate columns for period_of_record start and end
gfs = gfs[(gfs.status == 'ACTIVE')][keep_cols]

gfs['start_datetime'] = pd.to_datetime(gfs['period_of_record'].apply(lambda x: x['start']))
gfs['end_datetime'] = pd.to_datetime(gfs['period_of_record'].apply(lambda x: x['end']))
new_order = ["id", "stid", "name", "start_datetime", "end_datetime", "elevation", "station_info", "geometry"]
gfs = gfs[new_order]
gfs.head()

In [ ]:
gfs['marker-color'] = '#00FFFF' # cyan
gfs['marker-size'] = 'small'
gfs['marker-symbol'] = 'waterfall'

In [ ]:
# Save simplified, styled geojson
gfs.to_file("streamflow-stations-wa-styled.geojson", driver='GeoJSON')
!ls -tlrh *geojson

In [ ]:
# Combine both precip and streamflow
# gf = pd.concat([gfp, gfs], ignore_index=True)
# print('Total stations (precip + streamflow)=', len(gf))
# gf.head()
# gf.to_file('combined-stations-wa-styled.geojson', driver='GeoJSON')

## Seismic 

In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

In [ ]:
client = Client('IRIS')

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson"
)
minlon, minlat, maxlon, maxlat = aoi.total_bounds

In [ ]:
staqkwargs = {
    'channel': 'EHZ,HHZ,ENZ,HNZ,BNZ',
    'minlatitude': minlat,
    'minlongitude': minlon,
    'maxlongitude': maxlon,
    'maxlatitude': maxlat,
    'starttime': UTCDateTime('2025-01-01'),
    'endtime': UTCDateTime('2025-12-31'),
    'level': 'station'
}

# Query stations
inv = client.get_stations(**staqkwargs)

In [ ]:
len(inv.networks)

In [ ]:
nsll_set = set()
for net in inv.networks:
    for sta in net.stations:
        tup = (net.code,
               sta.code,
               sta.latitude,
               sta.longitude,
               sta.elevation,
               sta.start_date.datetime,
               sta.end_date.datetime if sta.end_date else None,
               sta.total_number_of_channels,
               sta.is_active(),
               f'https://ds.iris.edu/mda/{net.code}/{sta.code}'
        )
        nsll_set.add(tup)

df_seis = pd.DataFrame(list(nsll_set), columns=['network','station','latitude','longitude','elevation','start_datetime','end_datetime', 'total_number_of_channels', 'is_active', 'station_info'])
df_seis.head()

In [ ]:
print(len(df_seis), "active seismic stations in WA State")

In [ ]:
df_seis['marker-color'] = '#FF00FF' # magenta
df_seis['marker-size'] = 'small'
df_seis['marker-symbol'] = 'defibrillator'

In [ ]:
gf_seis = gpd.GeoDataFrame(df_seis, geometry=gpd.points_from_xy(df_seis.longitude, df_seis.latitude), crs='EPSG:4326')
#gf_seis.explore()

In [ ]:
gf_seis = gf_seis.drop(columns=['latitude','longitude','is_active'])
gf_seis.to_file('seismic-stations-wa-styled.geojson', driver='GeoJSON')

In [ ]:
# Ideally use same schema across data providers...
gf_seis.head()

In [ ]:

# loop over all the .geojson files in the PWD and combine all the features into a single combined.geojson. Just use the basic python file IO and JSON library for this
import json
import glob

# Find all .geojson files in the current directory, excluding the combined output
geojson_files = [f for f in glob.glob("*.geojson") if f != "combined-stations-wa-styled.geojson"]

all_features = []
crs =  "urn:ogc:def:crs:OGC:1.3:CRS84"

for fname in geojson_files:
    with open(fname, "r") as f:
        data = json.load(f)
        # Collect features
        if "features" in data:
            all_features.extend(data["features"])

with open("combined-stations-wa-styled.geojson", "w") as f:
    f.write('{"type": "FeatureCollection", ')
    f.write('"name": "stations-wa-styled", ')
    f.write('"crs": ' + json.dumps(crs) + ', ')
    f.write('"features": [\n')
    for i, feature in enumerate(all_features):
        f.write(json.dumps(feature))
        if i != len(all_features) - 1:
            f.write(",\n")
        else:
            f.write("\n")
    f.write("]}\n")

In [ ]:
# TODO: merge into single dataframe (careful with datetimes and schema)

# gf = pd.concat([gf, gf_seis], ignore_index=True)
# print('Total stations (precip + streamflow + seismic)=', len(gf))
# gf.to_file('combined-stations-wa-styled.geojson', driver='GeoJSON')